In [4]:
# %load build_financial_df.py
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import glob

#read file to access headers for the group of CSVs you want to use in DF
financial_headers = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/fin_head.csv', sep=",", header=None)
financial_headers_list = list(financial_headers.iloc[:,0])

#read all 20 txt files (from 2 directories) & concatenate to one df
path1 = r'/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/80to98'
path2 = r'/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/00to18'
files1 = glob.glob(path1 + "/*.txt")
files2 = glob.glob(path2 + "/*.txt")


li = []

for filename in files1:
    df = pd.read_csv(filename, index_col=None, header=None, sep='|', names=financial_headers_list, dtype={'DISTRICT':str})
    x = filename.replace('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/80to98/weball', '').replace(".txt",'')
    df['YEAR'] = int('19'+ x)
    li.append(df)
    
for filename in files2:
    df = pd.read_csv(filename, index_col=None, header=None, sep='|', names=financial_headers_list, dtype={'DISTRICT':str})
    x = filename.replace('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/00to18/weball', '').replace(".txt",'')
    df['YEAR'] = int('20'+ x)
    li.append(df)
    
financial_df_raw = pd.concat(li, axis=0, ignore_index=0)
financial_df_raw['LAST_NAME'] = financial_df_raw['NAME'].str.replace(',.*', '')
financial_df_raw["ELEC_ID"] = financial_df_raw['YEAR'].astype(str) + "-"+ financial_df_raw['STATE'].astype(str) +"-"+ financial_df_raw['DISTRICT'].astype(str)

'''
attempt to merge with candidate data files... changed mind for now
path = r'/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/cn'
files = glob.glob(path + "/*.txt")
cand_cols = ['ID', "x1", "x2", 'ELEC_YEAR',"x3",'OFFICE', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=None, sep='|', names=cand_cols)
    li.append(df)
    
#candidate_df_raw = pd.concat(li, axis=0, ignore_index=0)
#candidate_df = candidate_df_raw[['ID','ELEC_YEAR','OFFICE']]
#join1 = financial_df_raw.merge(candidate_df, how='inner', left_on='FEC_ID', right_on='ID')
#financial_df_plus = x123.drop('ID', axis=1)
'''

financial_df_raw["ELEC_ID"] = financial_df_raw['YEAR'].astype(str) + "-"+ financial_df_raw['STATE'].astype(str) +"-"+ financial_df_raw['DISTRICT'].astype(str)
#filter out presidential candidates
fin_cut_1 = financial_df_raw[financial_df_raw['STATE']!='00']
fin_cut_1['ELEC-CAND'] = fin_cut_1["ELEC_ID"].astype(str) +"-"+ fin_cut_1["LAST_NAME"].astype(str)
#filter out repeats in ELEC-CAND... may want to add some back because of last name issue
something = fin_cut_1['ELEC-CAND'].value_counts()
something
new_df = pd.DataFrame()
new_df['filtered'] = something[something > 1]
finance_df = fin_cut_1[~fin_cut_1['ELEC-CAND'].isin(new_df.reset_index()['index'])]
finance_df

'''
for later consideration :
mystery_df = fin_cut_1[fin_cut_1['ELEC-CAND'].isin(new_df.reset_index()['index'])]

'''


/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


"\nfor later consideration :\nmystery_df = fin_cut_1[fin_cut_1['ELEC-CAND'].isin(new_df.reset_index()['index'])]\n\n"

In [6]:
# %load build_results_df.py
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.style.use('ggplot')

results_data_76to2018 = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/results-from-Dataverse/house1976-2018.csv', sep=',', header=0)
results_data_80to18 = results_data_76to2018[results_data_76to2018['year']>1979]
senate_data = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/results-from-Dataverse/Senate/1976-2018-senate2.csv', sep=',', header=0)
senate_data_80to18 = senate_data[senate_data['year']>1979]

results_raw = pd.concat([senate_data_80to18, results_data_80to18], axis=0)

#filter out primaries
results_gen_elec1 = results_raw[results_raw['stage']=='gen']
#filter out write ins
results_gen_elec2 = results_gen_elec1[results_gen_elec1['writein']==False]
#filter out special
results_gen_elec3 = results_gen_elec2[results_gen_elec2['special']==False]
#add column for district string
results_gen_elec3['district_str'] = results_gen_elec3['district'].apply(lambda x: str(x) if len(str(x))>1 else '0' + str(x))
#add column for vote percentage
results_gen_elec3['vote%'] = 100 * results_gen_elec3['candidatevotes']/results_gen_elec3['totalvotes']
#add column for elec_id
results_gen_elec3['elec_id'] = results_gen_elec3['year'].astype(str) +'-'+results_gen_elec3['state_po']+'-'+results_gen_elec3['district_str']


#extract last name from candidate column (potential for error) & make new column
nix_name = ['Sr', 'Sr.', 'Jr', 'Jr.', 'III', 'II']
surname_series2 = results_gen_elec3['candidate'].apply(lambda x: x.split(' ')[-1].upper() if x.split(' ')[-1] not in nix_name else x.split(' ')[-2].upper())
#strip trailing commas 
results_gen_elec3['last_name'] = surname_series2.str.strip(',')
#look for faulty entries, particularly among frequently occurring names  

surname_series = results_gen_elec3['candidate'].apply(lambda x: x.split(' ')[-1] if x.split(' ')[-1] not in nix_name else x.split(' ')[-2])
# make list of all names that occur and remove bad entries ['Other', 'Vote/Scattering', 'scatter', 'Vote', ' ']
# *** ' ' = likely due to flaw in code
surname_lst = list(surname_series.value_counts().index)
last_name_list = [x.upper() for x in (surname_lst[3:7]+surname_lst[8:139] + surname_lst[140:])]
last_name_list
results_gen_elec3[results_gen_elec3['last_name'].isin(last_name_list)]
# make ELEC-CAND column (first edit elec_id to match formatting of finance_df)
results_gen_elec3['elec_id'] = results_gen_elec3['elec_id'].str.replace('statewide', '00')
results_gen_elec3['elec-cand'] = results_gen_elec3['elec_id'] + '-' + results_gen_elec3['last_name']
# remove duplicates from ELEC-CAND
repeats = results_gen_elec3['elec-cand'].value_counts()
repeats_df = pd.DataFrame()
repeats_df['filtered'] = repeats[repeats > 1]
mystery_repeats = results_gen_elec3[results_gen_elec3['elec-cand'].isin(repeats_df.reset_index()['index'])]
results_GE = results_gen_elec3[~results_gen_elec3['elec-cand'].isin(repeats_df.reset_index()['index'])]
# ready for merge? 
results_GE





/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

,candidate,candidatevotes,district,mode,office,party,runoff,special,stage,state,...,totalvotes,unofficial,version,writein,year,district_str,vote%,elec_id,last_name,elec-cand
270,Jim Folsom Jr.,610175,statewide,total,US Senate,democrat,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,47.053920,1980-AL-00,FOLSOM,1980-AL-00-FOLSOM
271,Jeremiah Denton,650362,statewide,total,US Senate,republican,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,50.152958,1980-AL-00,DENTON,1980-AL-00-DENTON
272,Sallie M. Hadnott,2973,statewide,total,US Senate,national democratic party of alabama,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,0.229264,1980-AL-00,HADNOTT,1980-AL-00-HADNOTT
273,William A. Crew,13098,statewide,total,US Senate,libertarian,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,1.010058,1980-AL-00,CREW,1980-AL-00-CREW
274,Michael R.A. Erdey,15989,statewide,total,US Senate,conservative,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,1.232999,1980-AL-00,ERDEY,1980-AL-00-ERDEY
275,Jim Partain,2649,statewide,total,US Senate,statesman,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,0.204279,1980-AL-00,PARTAIN,1980-AL-00-PARTAIN
276,Mohammed Oliver,1511,statewide,total,US Senate,socialist workers,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,0.116521,1980-AL-00,OLIVER,1980-AL-00-OLIVER
277,Clark S. Gruening,72007,statewide,total,US Senate,democrat,NaN,False,gen,Alaska,...,156762,False,20171011.0,False,1980,statewide,45.933964,1980-AK-00,GRUENING,1980-AK-00-GRUENING
279,Frank H. Murkowski,84159,statewide,total,US Senate,republican,NaN,False,gen,Alaska,...,156762,False,20171011.0,False,1980,statewide,53.685842,1980-AK-00,MURKOWSKI,1980-AK-00-MURKOWSKI
281,Josefina Otero,3266,statewide,total,US Senate,socialist workers,NaN,False,gen,Arizona,...,874238,False,20171011.0,False,1980,statewide,0.373582,1980-AZ-00,OTERO,1980-AZ-00-OTERO


In [10]:
merge_df_start = results_GE.merge(finance_df, how="inner", left_on="elec-cand", right_on="ELEC-CAND")

In [17]:
# filter out low vote% to limit to main candidates
merge_df_start[merge_df_start['vote%']>10]

x = finance_df['NAME'].str.contains('AYOTTE*', na=False)
finance_df[x]
'''
groupby each race (elec_id)
create a tuple of top two candidates from each race
create column for main_contenders and assign tuple to each member of group



'''

,candidate,candidatevotes,district,mode,office,party,runoff,special,stage,state,...,GEN_ELEC_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,YEAR,LAST_NAME,ELEC_ID,ELEC-CAND
0,Jim Folsom Jr.,610175,statewide,total,US Senate,democrat,NaN,False,gen,Alabama,...,47.0,0.00,0.00,12/31/1980,0.00,0.00,1980,FOLSOM,1980-AL-00,1980-AL-00-FOLSOM
1,Jeremiah Denton,650362,statewide,total,US Senate,republican,NaN,False,gen,Alabama,...,50.0,0.00,0.00,12/31/1980,0.00,0.00,1980,DENTON,1980-AL-00,1980-AL-00-DENTON
4,Clark S. Gruening,72007,statewide,total,US Senate,democrat,NaN,False,gen,Alaska,...,46.0,0.00,0.00,12/31/1980,0.00,0.00,1980,GRUENING,1980-AK-00,1980-AK-00-GRUENING
5,Frank H. Murkowski,84159,statewide,total,US Senate,republican,NaN,False,gen,Alaska,...,54.0,0.00,0.00,12/31/1980,0.00,0.00,1980,MURKOWSKI,1980-AK-00,1980-AK-00-MURKOWSKI
6,Barry Goldwater,432371,statewide,total,US Senate,republican,NaN,False,gen,Arizona,...,49.0,0.00,0.00,12/31/1980,0.00,0.00,1980,GOLDWATER,1980-AZ-00,1980-AZ-00-GOLDWATER
9,Bill Schulz,422972,statewide,total,US Senate,democrat,NaN,False,gen,Arizona,...,48.0,0.00,0.00,12/31/1980,0.00,0.00,1980,SCHULZ,1980-AZ-00,1980-AZ-00-SCHULZ
10,Dale Bumpers,477905,statewide,total,US Senate,democrat,NaN,False,gen,Arkansas,...,59.0,0.00,0.00,12/31/1980,0.00,0.00,1980,BUMPERS,1980-AR-00,1980-AR-00-BUMPERS
11,Bill Clark,330576,statewide,total,US Senate,republican,NaN,False,gen,Arkansas,...,41.0,0.00,0.00,12/31/1980,0.00,0.00,1980,CLARK,1980-AR-00,1980-AR-00-CLARK
12,Paul Gann,3091671,statewide,total,US Senate,republican,NaN,False,gen,California,...,37.0,0.00,0.00,12/31/1980,0.00,0.00,1980,GANN,1980-CA-00,1980-CA-00-GANN
15,Alan Cranston,4704098,statewide,total,US Senate,democrat,NaN,False,gen,California,...,57.0,0.00,0.00,12/31/1980,0.00,0.00,1980,CRANSTON,1980-CA-00,1980-CA-00-CRANSTON


In [23]:
#8783 races that survived the data cull
some = merge_df_start['elec_id'].value_counts()
#6547 races with 2+ candidates that survived the data cull
some[some>1]

2016-LA-00    15
2014-LA-06     8
1992-SD-00     8
2014-LA-05     7
2016-LA-03     7
2016-FL-00     7
2010-TN-05     7
2006-LA-02     6
2016-AK-00     6
1992-AK-00     6
1980-IL-00     6
2000-FL-00     6
2014-SD-00     6
2000-VT-00     6
1994-VT-00     6
2014-LA-00     6
2018-LA-03     6
2010-FL-00     6
1998-CO-00     5
1984-AK-00     5
2014-AK-00     5
2006-WA-00     5
2018-VT-00     5
2010-OH-00     5
2002-MN-00     5
2012-LA-02     5
2000-MI-00     5
2014-WY-00     5
2010-OR-00     5
1982-ND-00     5
              ..
2016-KY-02     1
1980-MA-03     1
1986-OK-04     1
2016-PA-02     1
1998-FL-23     1
2012-NM-03     1
1994-WA-08     1
1980-NY-23     1
1986-FL-15     1
1994-MI-14     1
1982-FL-10     1
1980-TX-11     1
1990-LA-00     1
1992-KS-02     1
2002-GA-04     1
1990-MD-02     1
1992-CA-18     1
2014-WA-01     1
2012-CA-18     1
1998-MI-16     1
2010-AL-04     1
2016-SC-01     1
1990-MI-13     1
1982-WI-07     1
1986-IL-01     1
2008-VT-00     1
2016-OH-05     1
1994-FL-08    